# Cat Dog Classifier

This is an introductory notebook to help you get into computer vision and deep learning. In this project we'll be training a model that can be used to classify images of cats and dogs. I initially worked on this project to get a better sense of the workflow for deep learning projects, and so I decided to explain it in the hope that it could be useful learning material for other people. Hope you find this interesting!

## Dataset
The dataset used here is a cleaned version of the Kaggle Cats and Dogs Dataset. The original dataset can be found here: https://bit.ly/2YTtAei . The original dataset had many corrupt images, so I removed them and provide the cleaned version here: https://bit.ly/3yGV4rm . Note that the file is a compressed .zip file that is 772 MB. The cleaned dataset a Cat folder with 12,471 images of cats and a Dog folder with 12,460 images of dogs.

## Libraries
For loading and processing images we'll be using OpenCV, and for the deep learning we'll use Keras. Keras is useful frontend API that uses the TensorFlow backend, which is a powerful machine learning platform developed by Google. We'll also be working with numpy arrays, and for saving/loading large variables for efficient reuse we'll use pickle files. Finally, we'll be making use of matplotlib for data visualisation.

In [ ]:
%matplotlib inline

import os
import tensorflow as tf
from tensorflow.keras import models, layers
import numpy as np
from cv2 import cv2 as cv
import pickle
from matplotlib import pyplot as plt
import random

If you have a CUDA-supported GPU, you can make use of the GPU version of tensorflow. You can check that tf is using a GPU using this print statement:

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

The first step is to compile our data into a form that can easily be used within our program. We do this using the `create_img_class_dataset` function shown below. The function takes compiles an input dataset for our classes based on the directory name (`img_dir`). It also takes in the image dimensions (`img_height`, `img_width`) as inputs, as well as the the `class_val` encoding value used for that class. In our case since we have only two classes, we'll be using 0 for cats and 1 for dogs.

The function works by looping through all of the image files in the specified directory, and applies some pre-processing on them. An image is first read using `cv.imread`, and then resized to the dimensions specified in the parameters. The image is then converted into a numpy array with a data type of `float32`. Next, we normalize all of the pixels so that their values are between 0 and 1, by dividing them by 255. Note that this is a scalar division that is applied to all of the RGB pixel components since this is a numpy array. Finally we prepend the class value to the image (so that we can keep track of it when we shuffle the images), and then we add the image to the dataset which is a list. Once all the processed images have been added, we return the dataset list.

In [ ]:
def create_img_class_dataset(img_dir, img_height, img_width, class_val):
    """Creates an image dataset for a class from a specified directory"""
    dataset = []
    for img_file in os.listdir(img_dir):
        img_path = os.path.join(img_dir, img_file)
        img = cv.imread(img_path)
        img = cv.resize(img, (img_height, img_width), interpolation=cv.INTER_AREA)
        img = np.array(img)
        img = img.astype('float32')
        img /= 255
        dataset.append([class_val, img])
    return dataset

In the next block, we use `create_img_class_dataset` to create datasets for our cat and dog classes. From these datasets, we can create sets for training and testing, namely `train_set` and `test_set` respectively.

One thing to note is that the processing that goes into generating `train_set` and `test_set` takes quite a while. To save time the next time we run this notebook, we can save these generated variables as pickle files, so instead of generating them again we can simply load them from the pickles. We use a try-except block to check whether the pickle files already exist, or whether we need to generate `train_set` and `test_set` from scratch.

In the case that pickles don't exist, we use `create_img_class_dataset` to generate cat and dog datasets, and then we shuffle these two datasets respectively. Note that we use a seed of 42 for the random number generator, so that our results are deterministic. We use a train-test split of 70:30, and so we split the cat and dog sets and then combine the training and testing sets respectively to obtain the `train_set` and `test_set` variables. These final variables are dumped into pickle files for convenient later reuse.

In [ ]:
IMG_HEIGHT = 64
IMG_WIDTH = 64

np.random.seed(42)
train_set = []
test_set = []

try:
    train_set = pickle.load(open("train_set.pickle", "rb"))
    test_set = pickle.load(open("test_set.pickle", "rb"))
    print("Datasets loaded from pickle files")
except:
    print("No pickle files found, compiling dataset variables")
    cat_set = create_img_class_dataset('kagglecatsanddogs_5340/PetImages/Cat', IMG_HEIGHT, IMG_WIDTH, 0)
    dog_set = create_img_class_dataset('kagglecatsanddogs_5340/PetImages/Dog', IMG_HEIGHT, IMG_WIDTH, 1)
    np.random.shuffle(cat_set)
    np.random.shuffle(dog_set)
    
    cat_train = cat_set[:int(len(cat_set)*0.7)]
    cat_test = cat_set[int(len(cat_set)*0.7):]
    
    dog_train = dog_set[:int(len(cat_set)*0.7)]
    dog_test = dog_set[int(len(cat_set)*0.7):]
    
    train_set = cat_train + dog_train
    test_set = cat_test + dog_test
    
    np.random.shuffle(train_set)
    np.random.shuffle(test_set)
    
    with open('train_set.pickle', 'wb') as handle:
        pickle.dump(train_set, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    with open('test_set.pickle', 'wb') as handle:
        pickle.dump(test_set, handle, protocol=pickle.HIGHEST_PROTOCOL)



From `train_set`, we want to seperate the labels from the data so that we can feed them as inputs into our model. Note that `train_set` is a 2D list with dimensions `n_images x 2`, with the first element representing the label and the second representing the image. After seperation, we convert `train_set_data` and `train_set_labels` into numpy arrays - which is the conventional iterable data structure for model fitting.

We do the same thing with `test_set`, in the block after it.

In [ ]:
train_set_data = []
train_set_labels = []

for i in range(len(train_set)):
    train_set_labels.append(train_set[i][0])
    train_set_data.append(train_set[i][1])

train_set_labels = np.array(train_set_labels)
train_set_data = np.array(train_set_data)

In [ ]:
test_set_data = []
test_set_labels = []

for i in range(len(test_set)):
    test_set_labels.append(test_set[i][0])
    test_set_data.append(test_set[i][1])

test_set_labels = np.array(test_set_labels)
test_set_data = np.array(test_set_data)

Now we come to the deep learning part of the project. The first step when it comes to deep learning tasks is to define the model architecture. In this case we use a Convolutional Neural Network (CNN), with 3 convolutional layers, 2 max pooling layers, and 2 final fully connected dense layers. A sequential type model is used, where each new layer is appended on to the existing network structure in a sequential fashion. The first layer of the network expects an input image of shape `[img_height, img_width, 3]`. Note that the last value represents the number of channels, which in this case is 3 for the red, green, and blue components of the image pixels.

The three convolutional layers of the network have 64, 128, 128 neurons respectively. We use the Rectified Linear Unit ('relu') as the activation function for these layers, in addition to the last fully connected dense layer which has 128 neurons. The stride for the convolutional layers and max pooling layers is `3x3`, and `2x2` respectively. The final layer consists of a single neuron with a sigmoid activation function, which will give us a classification value between 0 and 1.

Note that the model parameters can be played around with to observe differences in prediction accuracy and model performance. The structure used here achieves just almost 80% accuracy on the test set so it is far from optimal. Add/remove layers and neurons, change the activation function, and experiment with other types of layers provided by Keras!

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.summary()

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

We use the Adam optimizer, and Binary Cross Entropy for the loss function since we only have two classes.

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

The `fit` function is used to train the compiled model using an input dataset with its respective labels. The batch size is the number of training examples that are fed into the network before it is updated, and the number of epochs is the number of total passes of the whole dataset into the network. Again, these can be experimented with accordingly.

In [ ]:
history = model.fit(train_set_data, train_set_labels, batch_size=32, epochs=12, validation_data=(test_set_data, test_set_labels))

Once our model has been trained to a reasonable accuracy level, we can test out the predictions on random images in the test set. Run the following three blocks to see how the model performs on different images. Bear in mind, that it is not perfect and does make mistakes on some vague images!

In [ ]:
actual_labels = ['Cat', 'Dog']
rand_index = random.randint(0, len(test_set_data))
prediction_img = test_set_data[rand_index]
plt.imshow(prediction_img, interpolation='nearest')
prediction_img = prediction_img[None,:]
prediction = model.predict(prediction_img)

In [ ]:
print("Prediction:", actual_labels[round(prediction[0][0])], "- Value:", prediction[0][0])

In [ ]:
print("Actual label:", actual_labels[test_set_labels[rand_index]])

In this last block I've also tested the model on a picture of my cat Mishmish, just to make sure that he isn't actually dog or something...
Maybe try it out on your own pets if you want?

In [ ]:
mishmish_img = cv.imread("mishmish1.jpg")
mishmish_img = cv.resize(mishmish_img, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv.INTER_AREA)
mishmish_img = np.array(mishmish_img)
mishmish_img = mishmish_img.astype('float32')
mishmish_img /= 255
plt.imshow(mishmish_img, interpolation='nearest')
mishmish_img = mishmish_img[None,:]
mishmish_pred = model.predict(mishmish_img)
print("Prediction:", actual_labels[round(mishmish_pred[0][0])], "- Value:", mishmish_pred[0][0])